In [14]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_v1')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_v1


In [20]:
import numpy as np
from scipy.stats import norm
from scipy.io import savemat

#Parameters
b = 2 # Adjust as needed
h = 1 # Adjust as needed

elay = 3  # No. of shifts for decision
bandvec = [0.08]  # Optimal bandwidth for Gaussian kernel, adjust as needed
Blen = len(bandvec)


# Define the nvcost function
def nvcost(q, d, cu, co):
    if q < d:
        return cu * (d - q)  # Understocking cost
    else:
        return co * (q - d)  # Overstocking cost

# Extract the demand and features from the training and test sets
demand_train = np.hstack(target_train.values)
features_train = data_train.values
demand_test = np.hstack(target_test.values)
features_test = data_test.values

# Combine training and test sets for feature normalization and kernel calculation
combined_features = np.vstack((features_train, features_test))
combined_demand = np.hstack((demand_train, demand_test))

lntr = len(target_train)  # Training data size
lnte = len(target_test)  # Test data size
p = combined_features.shape[1]  # Number of features

delay = 3  # No. of shifts for decision
bandvec = [0.08]  # Optimal bandwidth for Gaussian kernel, adjust as needed

# Adjust the QfacD and Valfac arrays to store the results for the test set
QfacD = np.zeros((len(bandvec), lnte))
Valfac = np.zeros((len(bandvec), lnte))

# Combine training and test sets for feature normalization and kernel calculation
combined_features = np.vstack((features_train, features_test))
combined_demand = np.hstack((demand_train, demand_test))


for bi in range(Blen):
    bandwidth = bandvec[bi]

    for t in range(lntr, lntr + lnte):  
        FeaturesT = np.zeros_like(combined_features[t-lntr:t, :])

        for j in range(p):
            FeaturesT[t-lntr:t, j] = combined_features[t-lntr:t, j] / np.linalg.norm(combined_features[t-lntr:t, j], np.inf)

        kernel_weight = np.zeros(lntr)

        for i in range(lntr):
            kernel_weight[i-1] = norm.pdf(np.linalg.norm(FeaturesT[i-1, :] - FeaturesT[t-lntr+i-1, :]) / bandwidth)

        weights_normalisation_factor = np.sum(kernel_weight)
        kernel_cdf = np.zeros(lntr)

        sDemand, Map = np.sort(combined_demand[t-lntr+delay:t+delay-1]), np.argsort(combined_demand[t-lntr+delay:t+delay-1])
        skernel_weight = kernel_weight[Map]

        for i in range(lntr):
            ind_demand = sDemand <= sDemand[i-1]
            kernel_cdf[i-1] = np.sum(ind_demand * skernel_weight) / weights_normalisation_factor

        demand_range = combined_demand[t-lntr+delay:t+delay-1]
        q0 = demand_range[np.min(np.where(kernel_cdf >= b/(b+h)))]

        QfacD[bi, t-lntr] = q0
        Valfac[bi, t-lntr] = nvcost(q0, combined_demand[t+delay], b, h)


# The list of optimal order amounts for the test set
optimal_order_amounts = QfacD[0]


ValueError: could not broadcast input array from shape (208,) into shape (207,)

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.io import savemat

# Assuming 'Demand', 'TimeC', 'DayC', 'Past' are loaded from somewhere
# For demonstration, let's create some mock data
Demand = np.random.rand(1000)
TimeC = np.random.rand(1000, 1)  # Example feature
DayC = np.random.rand(1000, 1)  # Example feature
Past = np.random.rand(1000, 12)  # Past 12 days data

len_demand = len(Demand)
lntr = 12*7*16  # Training data size
lnva = lntr//2  # Validation data size
lnte = lnva
p = 12*14
delay = 3  # No. of shifts for decision
bandvec = [0.08]  # Optimal bandwidth for Gaussian kernel, adjust as needed

b = 2.5 / 3.5
h = 1 / 3.5
r = b / (b + h)

# Feature selection based on 'p' value, adjust 'p' and 'Features' as needed
if p == -36:
    Features = TimeC
elif p == -24:
    Features = DayC
elif p == -12:
    Features = np.hstack([DayC, TimeC])
elif p == 0:
    Features = np.hstack([DayC, TimeC, TimeC])  # 'Time' should be another feature array
else:
    Features = np.hstack([DayC, TimeC, TimeC, Past[:, :p]])

QfacD = np.zeros((len(bandvec), lnva))
Valfac = np.zeros((len(bandvec), lnva))

for bi, bandwidth in enumerate(bandvec):
    for t in range(lntr + lnva + 1, lntr + lnva + 101):  # Adjust the range as needed
        FeaturesT = Features / np.linalg.norm(Features, np.inf, axis=0)

        kernel_weight = np.array([norm.pdf(np.linalg.norm(FeaturesT[t, :] - FeaturesT[t-lntr+i-1, :]) / bandwidth) 
                                  for i in range(1, lntr + 1)])
        weights_normalisation_factor = np.sum(kernel_weight)
        
        sorted_demand, Map = zip(*sorted((d, i) for i, d in enumerate(Demand[t-lntr+delay-1:t+delay])))
        sorted_kernel_weight = np.array([kernel_weight[i] for i in Map])
        
        kernel_cdf = np.cumsum(sorted_kernel_weight[sorted_demand <= sorted_demand[i]] / weights_normalisation_factor 
                               for i in range(lntr))
        
        demand_range = Demand[t-lntr+delay-1:t+delay]
        q0 = demand_range[np.argmax(kernel_cdf >= r)]
        
        QfacD[bi, t-lntr-lnva-1] = q0
        # Valfac calculation requires `nvcost` function definition

# Save results (adjust 'loc' path and file name as needed)
savemat('nv_kernelG_results.mat', {'QfacD': QfacD})


IndexError: index 2017 is out of bounds for axis 0 with size 1000